In [1]:
import json
import os
from tqdm import tqdm
from multiprocessing import Pool
import sys
import time
sys.path.append('/home/sreenath_a/Projects/OSS/Production/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect

In [2]:
cpu_count = os.cpu_count()
print(cpu_count)

20


In [3]:
columns = ['owner', 'repo', 'name', 'description', 'licenseName', 'licenseSpdxId', 'licenseUrl',
           'shortDescriptionHTML', 'repourl', 'createdAt', 'updatedAt', 'pushedAt',
           'forkCount', 'stargazerCount', 'issuesCount', 'pullRequestsCount']

In [4]:
# parallelcount = cpu_count
parallelcount = 8

In [5]:
#Load the json file
with open('/home/sreenath_a/Projects/OSS/API_keys.json') as f:
    keys = json.load(f)['keys']

In [6]:
#create an empty dataframe and save as csv
if not os.path.exists('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv'):
    df = pd.DataFrame(columns=columns)
    df.to_csv('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv', index=False)

In [7]:
#get all the packages alrady collected
slugs_collected = []
for df in pd.read_csv('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv', chunksize=1000):
    repo_names = df['repo'].tolist()
    repo_owners = df['owner'].tolist()
    for i in range(len(repo_names)):
        slugs_collected.append(str(repo_owners[i].strip()) + '/' + str(repo_names[i].strip()))
    slugs_collected = [i.strip() for i in list(set(slugs_collected))]

In [8]:
def get_owner_and_repo(url):
    ### if it has .git, remove it
    if url.endswith(".git"):
        url = url[:-4]
    ### get owner and repo
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [9]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

In [10]:
def get_repo_info(owner, repo, key=None):
    if owner + '/' + repo in slugs_collected:
        return None
    repo_info = collector.getRepoInfo(owner, repo, key)
    del repo_info['readme']
    return repo_info

In [14]:
def get_owner_and_repo(url):
    if type(url) != str:
        return None, None
    if "/" not in url:
        return None, None
    url = str(url).strip()
    if url.endswith(".git"):
        url = url[:-4]
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [15]:
df_packages = pd.read_csv("/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/R_data_collection/cran_github_all_slugs.csv")[['Package', 'Slugs', 'URL']]
total_pacakges = pd.read_csv("/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/R_data_collection/cran_github_all_slugs.csv").shape[0]
print(total_pacakges)

7648


In [16]:
for df in tqdm(pd.read_csv("/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/R_data_collection/cran_github_all_slugs.csv", chunksize=parallelcount), total=total_pacakges//parallelcount):
    slugs = [get_owner_and_repo(i) for i in df.Slugs]
    slugs = [i for i in slugs if i[0] is not None]
    owners = [i[0] for i in slugs]
    repos = [i[1] for i in slugs]
    best_keys = collector.getBestKeys(n=parallelcount)
    args = list(zip(owners, repos, best_keys))
    with Pool(parallelcount) as p:
        result = p.starmap(get_repo_info, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    result_df = pd.DataFrame.from_dict(result, orient='columns')
    result_df.to_csv('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv', mode='a', header=False, index=False)

100%|██████████| 956/956 [05:18<00:00,  3.00it/s]


In [17]:
df_repos = pd.read_csv('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv')
df_repos = df_repos[df_repos.owner.notna()].reset_index(drop=True)

In [18]:
df_repos.shape

(7602, 16)

In [19]:
df_repos.to_csv('/home/sreenath_a/Projects/OSS/Production/Cran_GH_Data_Repos.csv', index=False)